In [1]:
import pandas as pd
import json
import os
import context_window_helper as cwh

In [2]:
filepath = r"C:\Users\gluzk\source\repos\CS2_cheat_detection\Data_Conversion_Pipeline\test_data\0"

# Loading csv tick data as a pd.DataFrame
match_ticks = pd.read_parquet(path=filepath+".parquet")

# Loading json event data a list of tuples (str, pd.Dataframe)
def json_2_eventlist(filepath:str) -> list:   
    with open(filepath, "r") as f:
        json_data = json.load(f)

    data = []       

    for key, value in json_data.items():
        if isinstance(value, list):
            df = pd.DataFrame(value)
            data.append((key, df))

    return data

match_events = json_2_eventlist(filepath=filepath+".json")

In [3]:
ticks_before_kill = 896
tick_after_kill = 128

context_window_vals = ["attacker_X", "attacker_Y", "attacker_Z", "attacker_vel_X", "attacker_vel_Y", "attacker_vel_Z", "attacker_pitch", "attacker_yaw", "attacker_pitch_delta", 
                       "attacker_yaw_delta", "attacker_pitch_head_delta", "attacker_yaw_head_delta", "attacker_flashed", "attacker_shot", "attacker_kill", "is_kill_through_smoke", 
                       "is_kill_wallbang", "attacker_midair", "attacker_weapon_knife", "attacker_weapon_auto_rifle", "attacker_weapon_semi_rifle", "attacker_weapon_pistol",
                       "attacker_weapon_grenade", "attacker_weapon_smg", "attacker_weapon_shotgun",
                       "victim_X", "victim_Y", "victim_Z", "victim_health", "victim_noise", "map_dust2", "map_mirage", "map_inferno", "map_train",
                       "map_nuke", "map_ancient", "map_vertigo", "map_anubis", "map_office", "map_overpass", "map_basalt", "map_edin", "map_italy", "map_thera", "map_mills"]

In [4]:
len(context_window_vals)

45

In [5]:
list(match_ticks.columns)

['inventory',
 'usercmd_input_history',
 'inventory_as_ids',
 'approximate_spotted_by',
 'aim_punch_angle_vel',
 'aim_punch_angle',
 'active_weapon_ammo',
 'item_def_idx',
 'weapon_quality',
 'entity_lvl',
 'item_id_high',
 'item_id_low',
 'item_account_id',
 'inventory_position',
 'is_initialized',
 'econ_item_attribute_def_idx',
 'initial_value',
 'refundable_currency',
 'set_bonus',
 'fire_seq_start_time',
 'fire_seq_start_time_change',
 'm_iState',
 'weapon_mode',
 'accuracy_penalty',
 'i_recoil_idx',
 'fl_recoil_idx',
 'is_burst_mode',
 'is_in_reload',
 'reload_visually_complete',
 'dropped_at_time',
 'is_hauled_back',
 'is_silencer_on',
 'time_silencer_switch_complete',
 'orig_team_number',
 'prev_owner',
 'last_shot_time',
 'iron_sight_mode',
 'num_empty_attacks',
 'zoom_lvl',
 'burst_shots_remaining',
 'needs_bolt_action',
 'next_primary_attack_tick',
 'next_primary_attack_tick_ratio',
 'next_secondary_attack_tick',
 'next_secondary_attack_tick_ratio',
 'total_ammo_left',
 'is_

In [6]:
match_ticks

,inventory,usercmd_input_history,inventory_as_ids,approximate_spotted_by,aim_punch_angle_vel,aim_punch_angle,active_weapon_ammo,item_def_idx,weapon_quality,entity_lvl,...,usercmd_forward_move,usercmd_consumed_server_angle_changes,is_alive,velocity_Z,active_weapon_name,usercmd_mouse_dx,game_time,tick,steamid,name
0,"[knife_t, Glock-18]",[],"[59, 4]",[],"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",0.0,59.0,0.0,1.0,...,NaN,NaN,True,NaN,knife_t,NaN,51.187500,1,Player_8,Player_8
1,"[knife_t, Glock-18, AK-47]",[],"[59, 4, 7]",[],"[0.0, 0.0, 0.0]","[0.4380251467227936, 0.01679001934826374, 0.0]",30.0,7.0,4.0,1.0,...,NaN,NaN,True,NaN,AK-47,NaN,51.187500,1,Player_9,Player_9
2,"[knife_t, Glock-18]",[],"[59, 4]",[],"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",20.0,4.0,4.0,1.0,...,NaN,NaN,True,NaN,Glock-18,NaN,51.187500,1,Player_7,Player_7
3,"[knife, USP-S]",[],"[42, 61]",[],"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",0.0,42.0,0.0,1.0,...,NaN,NaN,True,NaN,knife,NaN,51.187500,1,Player_4,Player_4
4,[],[],[],[Player_3],"[0.0, 0.0, 0.0]","[-36.0, 0.0, -27.0]",NaN,NaN,NaN,NaN,...,NaN,NaN,False,NaN,None,NaN,51.187500,1,Player_10,Player_10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601785,[],"[{'player_tick_count': 63464, 'player_tick_fra...",[],[],"[0.0, 0.0, 0.0]","[-0.6168899536132812, 0.0, 0.0]",NaN,NaN,NaN,NaN,...,0.0,0.0,False,0.0,None,0.0,991.703125,60194,Player_10,Player_10
601786,[],"[{'player_tick_count': 58461, 'player_tick_fra...",[],[],"[0.0, 0.0, 0.0]","[-4.232666969299316, 0.0, 0.0]",NaN,NaN,NaN,NaN,...,0.0,0.0,False,0.0,None,0.0,991.703125,60194,Player_1,Player_1
601787,[],"[{'player_tick_count': 63357, 'player_tick_fra...",[],[],"[-149.81370544433594, 14.492167472839355, 0.0]","[0.0, 0.0, 0.0]",NaN,NaN,NaN,NaN,...,0.0,0.0,False,0.0,None,0.0,991.703125,60194,Player_5,Player_5
601788,[],"[{'player_tick_count': 59411, 'player_tick_fra...",[],[],"[-2.8513312339782715, -0.37512412667274475, 0.0]","[-36.0, 0.0, -27.0]",NaN,NaN,NaN,NaN,...,0.0,0.0,False,0.0,None,0.0,991.703125,60194,Player_2,Player_2


In [ ]:
player = "Player_3"
player_death_idx = -1


for idx, event in enumerate(match_events):
    if event[0] == "player_death":
        player_death_idx = idx
        break


player_deaths = cwh.get_player_kills(match_events, player, player_death_idx)
start_ticks, end_ticks = cwh.get_context_window_ticks(player_deaths, ticks_before_kill, tick_after_kill)

context_window = pd.DataFrame(columns=context_window_vals)

for i in range(len(start_ticks)):
    # X_, Y_, Z_ = cwh.get_attacker_XYZ(start_ticks[i], end_ticks[i], match_ticks, player)
    # context_window["attacker_X"] = X_
    # context_window["attacker_Y"] = Y_
    # context_window["attacker_Z"] = Z_
    context_window["attacker_X"] = cwh.get_value_from_match_data("X", start_ticks[i], end_ticks[i], match_ticks, player)
    context_window["attacker_Y"] = cwh.get_value_from_match_data("Y", start_ticks[i], end_ticks[i], match_ticks, player)
    context_window["attacker_Z"] = cwh.get_value_from_match_data("Z", start_ticks[i], end_ticks[i], match_ticks, player)



    # vel_X, vel_Y, vel_Z = cwh.get_attacker_vels(start_ticks[i], end_ticks[i], match_ticks, player)
    # context_window["attacker_vel_X"] = vel_X
    # context_window["attacker_vel_Y"] = vel_Y
    # context_window["attacker_vel_Z"] = vel_Z
    break # Remove later, currently used for testing
context_window


,attacker_X,attacker_Y,attacker_Z,attacker_vel_X,attacker_vel_Y,attacker_vel_Z,attacker_pitch,attacker_yaw,attacker_pitch_delta,attacker_yaw_delta,...,map_ancient,map_vertigo,map_anubis,map_office,map_overpass,map_basalt,map_edin,map_italy,map_thera,map_mills
0,-1776.000000,-1976.000000,-263.96875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1776.000000,-1976.000000,-263.96875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1776.000000,-1976.000000,-263.96875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1776.000000,-1976.000000,-263.96875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1776.000000,-1976.000000,-263.96875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,-639.242798,-1500.297852,-167.96875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020,-639.003296,-1496.398926,-167.96875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1021,-638.776245,-1492.499268,-167.96875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1022,-638.560852,-1488.598999,-167.96875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
